In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_functions import seqsignet_hyperparameter_search

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-09-28 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-28 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-28 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-28 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-28 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
lstm_hidden_dim_sizes = [384]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

# history_length=11

In [14]:
shift = 3
window_size = 5
n = 3

In [15]:
(
    seqsignet_network_umap_kfold_11,
    best_seqsignet_network_umap_kfold_11,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_3_kfold_best_model.csv


In [16]:
seqsignet_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.638889,0.508545,"[0.7702565589215828, 0.40689325035902346, 0.34...",0.512418,"[0.7721592560302237, 0.4846066134549601, 0.280...",0.526341,"[0.7683632157316368, 0.35066006600660066, 0.46]",None,0.666346,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.607854,0.508247,"[0.7381772627622926, 0.4333199839807769, 0.353...",0.499695,"[0.8049863387978142, 0.421011673151751, 0.2730...",0.542659,"[0.6816078658183922, 0.44636963696369636, 0.5]",None,0.644188,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.634866,0.512799,"[0.7605220706848511, 0.42493415276558383, 0.35...",0.508132,"[0.771496578399286, 0.4540337711069418, 0.2988...",0.526701,"[0.7498554077501446, 0.39933993399339934, 0.43...",None,0.670520,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.632759,0.525914,"[0.7523186863311541, 0.46548323471400394, 0.35...",0.515452,"[0.7932029496633536, 0.4459561602418745, 0.307...",0.545596,"[0.7154424522845575, 0.4867986798679868, 0.434...",None,0.667470,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.628544,0.511676,"[0.7561487702459508, 0.42851516345017715, 0.35...",0.503852,"[0.7853582554517134, 0.40994724943481536, 0.31...",0.523535,"[0.7290341237709659, 0.44884488448844884, 0.39...",None,0.663777,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.646935,0.525294,"[0.769569687178734, 0.4446280991735537, 0.3616...",0.519512,"[0.7818561623396001, 0.445364238410596, 0.3313...",0.533247,"[0.7576633892423366, 0.4438943894389439, 0.398...",None,0.667791,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.633716,0.514062,"[0.7607434944237919, 0.4357262103505843, 0.345...",0.506783,"[0.7829813284358739, 0.4408783783783784, 0.296...",0.528324,"[0.739733950260266, 0.4306930693069307, 0.4145...",None,0.672768,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.636015,0.517543,"[0.7628957930726922, 0.44041450777202074, 0.34...",0.511661,"[0.7849495258488834, 0.46195652173913043, 0.28...",0.535492,"[0.7420474262579526, 0.4207920792079208, 0.443...",None,0.662171,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.625479,0.518182,"[0.7538626281168731, 0.4445371142618849, 0.356...",0.510099,"[0.8002598246183826, 0.4494097807757167, 0.280...",0.546531,"[0.7125506072874493, 0.4397689768976898, 0.487...",None,0.648523,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [17]:
best_seqsignet_network_umap_kfold_11["f1"].mean()

0.5207346518508088

In [18]:
best_seqsignet_network_umap_kfold_11["precision"].mean()

0.5127868034452244

In [19]:
best_seqsignet_network_umap_kfold_11["recall"].mean()

0.5373551744995205

In [20]:
np.stack(best_seqsignet_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75825457, 0.44574979, 0.3581996 ])

In [21]:
np.stack(best_seqsignet_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78626238, 0.444712  , 0.30738603])

In [22]:
np.stack(best_seqsignet_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.73231155, 0.44884488, 0.43090909])

## GRP

In [23]:
(
    seqsignet_network_grp_kfold_11,
    best_seqsignet_network_grp_kfold_11,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_3_kfold_best_model.csv


In [24]:
seqsignet_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.638506,0.511102,"[0.768158047646717, 0.4081055607917059, 0.3570...",0.512387,"[0.7717454757734968, 0.4758241758241758, 0.289...",0.529106,"[0.7646038172353962, 0.3572607260726073, 0.465...",None,0.662813,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.610920,0.508169,"[0.737459484488347, 0.42924159499609066, 0.357...",0.497643,"[0.7907977490897053, 0.4078751857355126, 0.294...",0.533399,"[0.6908617698091383, 0.452970297029703, 0.4563...",None,0.653500,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.635441,0.518679,"[0.7595684941628491, 0.42942686056458507, 0.36...",0.511533,"[0.7766696887277122, 0.44582593250444047, 0.31...",0.534283,"[0.7432041642567958, 0.4141914191419142, 0.445...",None,0.665382,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.633908,0.515753,"[0.7583135391923991, 0.4389837567680134, 0.349...",0.508384,"[0.7791336180597925, 0.4432296047098402, 0.302...",0.529314,"[0.7385772122614228, 0.43481848184818483, 0.41...",None,0.669236,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.638314,0.516771,"[0.7636256794476275, 0.4289383561643835, 0.357...",0.510786,"[0.7760525530008958, 0.445729537366548, 0.3105...",0.528925,"[0.7515905147484094, 0.41336633663366334, 0.42...",None,0.671805,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.620306,0.508861,"[0.7464042392127176, 0.43509906995551967, 0.34...",0.499780,"[0.7832856688910073, 0.42664551942902457, 0.28...",0.528002,"[0.7128397917871602, 0.4438943894389439, 0.427...",None,0.662171,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.637739,0.515208,"[0.7623820754716981, 0.4396155453405766, 0.343...",0.509081,"[0.777510523150932, 0.445385266723116, 0.30434...",0.525457,"[0.7478311162521689, 0.43399339933993397, 0.39...",None,0.671805,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.628352,0.503916,"[0.7560046865846515, 0.41318681318681316, 0.34...",0.499666,"[0.7658753709198813, 0.44214487300094074, 0.29...",0.516846,"[0.7463851937536148, 0.38778877887788776, 0.41...",None,0.664900,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.627011,0.519737,"[0.7545440659844203, 0.44472049689440996, 0.35...",0.510878,"[0.7996115247652962, 0.4463840399002494, 0.286...",0.546997,"[0.7142857142857143, 0.4430693069306931, 0.483...",None,0.655427,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [25]:
best_seqsignet_network_grp_kfold_11["f1"].mean()

0.5190070840121788

In [26]:
best_seqsignet_network_grp_kfold_11["precision"].mean()

0.5109347779258603

In [27]:
best_seqsignet_network_grp_kfold_11["recall"].mean()

0.5351426692698188

In [28]:
np.stack(best_seqsignet_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.75771302, 0.44357586, 0.35573237])

In [29]:
np.stack(best_seqsignet_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.78620464, 0.43984308, 0.30675661])

In [30]:
np.stack(best_seqsignet_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.7313476 , 0.4480198 , 0.42606061])

# history_length=20

In [10]:
shift = 3
window_size = 5
n = 6

In [32]:
(
    seqsignet_network_umap_kfold_20,
    best_seqsignet_network_umap_kfold_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_6_kfold_best_model.csv


In [33]:
seqsignet_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.645594,0.517765,"[0.7717375630857967, 0.41417910447761197, 0.36...",0.518095,"[0.7696289905090595, 0.47639484978540775, 0.30...",0.531580,"[0.7738577212261423, 0.36633663366336633, 0.45...",None,0.671162,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.617433,0.508190,"[0.7477380769820579, 0.43321299638989175, 0.34...",0.499059,"[0.7959516813581456, 0.4215456674473068, 0.279...",0.532010,"[0.7050318102949682, 0.44554455445544555, 0.44...",None,0.658157,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.637931,0.517992,"[0.7646972152644762, 0.42429501084598703, 0.36...",0.511726,"[0.7795133673775909, 0.4473924977127173, 0.308...",0.533724,"[0.7504337767495662, 0.4034653465346535, 0.447...",None,0.666185,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.632950,0.520069,"[0.7534391534391535, 0.46549707602339185, 0.34...",0.511139,"[0.7893569844789357, 0.44124168514412415, 0.30...",0.534710,"[0.7206477732793523, 0.49257425742574257, 0.39...",None,0.673089,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.632184,0.518627,"[0.7546205860255446, 0.4435082343929529, 0.357...",0.511943,"[0.7854238348451673, 0.413867047891351, 0.3365...",0.528561,"[0.7261422787738577, 0.4777227722772277, 0.381...",None,0.671965,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.642146,0.515077,"[0.7689614935822638, 0.4226415094339622, 0.353...",0.510691,"[0.7757504414361389, 0.4296675191815857, 0.326...",0.521195,"[0.7622903412377097, 0.4158415841584158, 0.385...",None,0.674374,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.637548,0.518798,"[0.764128494943486, 0.4386989157631359, 0.3535...",0.511161,"[0.786589099816289, 0.44350758853288363, 0.303...",0.533515,"[0.742914979757085, 0.43399339933993397, 0.423...",None,0.668593,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.625670,0.520733,"[0.7467315293402251, 0.4520884520884521, 0.363...",0.510837,"[0.7871794871794872, 0.44878048780487806, 0.29...",0.544925,"[0.7102371312897628, 0.45544554455445546, 0.46...",None,0.655267,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.625096,0.518500,"[0.7538626281168731, 0.4389632107023411, 0.362...",0.510098,"[0.8002598246183826, 0.4449152542372881, 0.285...",0.547967,"[0.7125506072874493, 0.43316831683168316, 0.49...",None,0.656069,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [34]:
best_seqsignet_network_umap_kfold_20["f1"].mean()

0.5164772721152695

In [35]:
best_seqsignet_network_umap_kfold_20["precision"].mean()

0.5081440744623626

In [36]:
best_seqsignet_network_umap_kfold_20["recall"].mean()

0.5342635294954211

In [37]:
np.stack(best_seqsignet_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.7576027 , 0.43787008, 0.35395904])

In [38]:
np.stack(best_seqsignet_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78616118, 0.4405165 , 0.29775454])

In [39]:
np.stack(best_seqsignet_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.73105842, 0.43536854, 0.43636364])

## GRP

In [11]:
(
    seqsignet_network_grp_kfold_20,
    best_seqsignet_network_grp_kfold_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_6_kfold_best_model.csv


In [12]:
seqsignet_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.645019,0.518354,"[0.7737332556785089, 0.4254324450677887, 0.355...",0.519599,"[0.7791788856304985, 0.49083063646170444, 0.28...",0.535804,"[0.7683632157316368, 0.37541254125412543, 0.46...",None,0.665061,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.622989,0.517147,"[0.7504581551618815, 0.4424320827943079, 0.358...",0.511407,"[0.7951456310679612, 0.4634146341463415, 0.275...",0.548840,"[0.7105263157894737, 0.42326732673267325, 0.51...",None,0.647720,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.634674,0.516693,"[0.7589773902763411, 0.4318568994889267, 0.359...",0.509822,"[0.7760652765185857, 0.44630281690140844, 0.30...",0.531223,"[0.7426257952573742, 0.4183168316831683, 0.432...",None,0.667470,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.635249,0.518723,"[0.7579542075527802, 0.445364238410596, 0.3528...",0.510994,"[0.7799877600979193, 0.446843853820598, 0.3061...",0.532463,"[0.7371312897628687, 0.4438943894389439, 0.416...",None,0.668593,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.637356,0.516567,"[0.7628865979381444, 0.4306506849315068, 0.356...",0.510368,"[0.7773109243697479, 0.447508896797153, 0.3062...",0.529820,"[0.7489878542510121, 0.415016501650165, 0.4254...",None,0.673089,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.619923,0.508384,"[0.7460606060606062, 0.43593875906526997, 0.34...",0.499309,"[0.7835773392743476, 0.4259842519685039, 0.288...",0.527326,"[0.7119722382880278, 0.44636963696369636, 0.42...",None,0.661047,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.632759,0.510989,"[0.7589272484812565, 0.43378433784337844, 0.34...",0.504245,"[0.7781829231236707, 0.4311328443357783, 0.303...",0.521448,"[0.7406015037593985, 0.43646864686468645, 0.38...",None,0.671805,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.634866,0.515822,"[0.760064887184781, 0.430257979886314, 0.35714...",0.510411,"[0.7755040625940415, 0.4576744186046512, 0.298...",0.532208,"[0.7452284557547716, 0.40594059405940597, 0.44...",None,0.662974,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.632184,0.521447,"[0.759119686463672, 0.43986254295532645, 0.365...",0.514184,"[0.7928211586901763, 0.45878136200716846, 0.29...",0.547173,"[0.7281665702718334, 0.42244224422442245, 0.49...",None,0.657514,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [13]:
best_seqsignet_network_grp_kfold_20["f1"].mean()

0.5191028390785046

In [14]:
best_seqsignet_network_grp_kfold_20["precision"].mean()

0.5111709036733086

In [15]:
best_seqsignet_network_grp_kfold_20["recall"].mean()

0.5346424414949689

In [16]:
np.stack(best_seqsignet_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.75719067, 0.44414943, 0.35596842])

In [17]:
np.stack(best_seqsignet_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.78400753, 0.44155159, 0.3079536 ])

In [18]:
np.stack(best_seqsignet_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.73221515, 0.44746975, 0.42424242])

# history_length=35

In [13]:
shift = 3
window_size = 5
n = 11

In [20]:
(
    seqsignet_network_umap_kfold_35,
    best_seqsignet_network_umap_kfold_35,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_11_kfold_best_model.csv


In [21]:
seqsignet_network_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.639655,0.517962,"[0.7670387779083432, 0.4291571753986333, 0.357...",0.516096,"[0.7794029850746269, 0.47914547304170907, 0.28...",0.536982,"[0.7550607287449392, 0.3886138613861386, 0.467...",None,0.664419,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.621456,0.512404,"[0.749539028887523, 0.4483985765124555, 0.3392...",0.503070,"[0.799672131147541, 0.4305239179954442, 0.2790...",0.535290,"[0.705320994794679, 0.46782178217821785, 0.432...",None,0.655106,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.642337,0.521155,"[0.7659387366261177, 0.4381285526891123, 0.359...",0.516327,"[0.7765230312035661, 0.466046511627907, 0.3064...",0.534517,"[0.7556390977443609, 0.41336633663366334, 0.43...",None,0.669075,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.637356,0.520622,"[0.7598620068996551, 0.46184578505793045, 0.34...",0.512352,"[0.7893424742910564, 0.447714949651433, 0.3]",0.534043,"[0.7325043377674957, 0.4768976897689769, 0.392...",None,0.669717,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.622797,0.503444,"[0.7505285412262156, 0.43781094527363185, 0.32...",0.496620,"[0.7853982300884956, 0.40827980014275517, 0.29...",0.514433,"[0.7186234817813765, 0.47194719471947194, 0.35...",None,0.669717,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.640613,0.516080,"[0.7657644476956841, 0.43075615972812237, 0.35...",0.510915,"[0.7749481788569736, 0.44395796847635727, 0.31...",0.525038,"[0.7567958357432042, 0.4183168316831683, 0.4]",None,0.672447,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.636398,0.514250,"[0.7625129956928561, 0.4431771894093686, 0.337...",0.507346,"[0.7838167938931297, 0.43765084473049076, 0.30...",0.524939,"[0.7423366107576634, 0.44884488448844884, 0.38...",None,0.668433,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.639080,0.513676,"[0.7629803461425638, 0.43858202802967855, 0.33...",0.508856,"[0.7741071428571429, 0.4382207578253707, 0.314...",0.520068,"[0.7521688837478311, 0.4389438943894389, 0.369...",None,0.669717,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.631226,0.520102,"[0.7595013681970204, 0.4559322033898305, 0.344...",0.513780,"[0.8006410256410257, 0.4686411149825784, 0.272...",0.545729,"[0.7223828802776171, 0.4438943894389439, 0.470...",None,0.652537,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [22]:
best_seqsignet_network_umap_kfold_35["f1"].mean()

0.5161101406793733

In [23]:
best_seqsignet_network_umap_kfold_35["precision"].mean()

0.5106495948435748

In [24]:
best_seqsignet_network_umap_kfold_35["recall"].mean()

0.5328843622746696

In [25]:
np.stack(best_seqsignet_network_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.76180258, 0.43455228, 0.35197556])

In [26]:
np.stack(best_seqsignet_network_umap_kfold_35["precision_scores"]).mean(axis=0)

array([0.78263912, 0.45207925, 0.29723042])

In [27]:
np.stack(best_seqsignet_network_umap_kfold_35["recall_scores"]).mean(axis=0)

array([0.74204743, 0.42024202, 0.43636364])

## GRP

In [14]:
(
    seqsignet_network_grp_kfold_35,
    best_seqsignet_network_grp_kfold_35,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_11_kfold_best_model.csv


In [15]:
seqsignet_network_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.629502,0.510676,"[0.7568848090020728, 0.422707423580786, 0.3524...",0.505082,"[0.7754854368932039, 0.4489795918367347, 0.290...",0.528589,"[0.7391555812608445, 0.39933993399339934, 0.44...",None,0.666185,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.621648,0.521669,"[0.7463700957676861, 0.4457096380642538, 0.372...",0.511381,"[0.8010610079575596, 0.43945469125902165, 0.29...",0.553908,"[0.6986697513013302, 0.4521452145214521, 0.510...",None,0.647720,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.626628,0.509915,"[0.7536016634486855, 0.4194659776055125, 0.356...",0.503068,"[0.7746564885496183, 0.43873873873873875, 0.29...",0.528189,"[0.7336610757663389, 0.4018151815181518, 0.449...",None,0.663295,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.635824,0.521434,"[0.7582286056253741, 0.45010101010101006, 0.35...",0.512803,"[0.7854928704277744, 0.4410134600158353, 0.311...",0.535637,"[0.7327935222672065, 0.45957095709570955, 0.41...",None,0.670520,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.644061,0.521242,"[0.7690956979806848, 0.4326672458731538, 0.361...",0.516105,"[0.7784360189573459, 0.45688073394495415, 0.31...",0.533320,"[0.7599768652400232, 0.41089108910891087, 0.42...",None,0.672929,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.635249,0.519075,"[0.7598031906962874, 0.4433333333333333, 0.354...",0.511153,"[0.7842413050169282, 0.4478114478114478, 0.301...",0.534959,"[0.7368421052631579, 0.4389438943894389, 0.429...",None,0.668754,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.625287,0.509412,"[0.7515351205631272, 0.43060643060643056, 0.34...",0.501092,"[0.7794346070208139, 0.4248995983935743, 0.298...",0.524314,"[0.7255639097744361, 0.43646864686468645, 0.41...",None,0.669075,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.627586,0.518193,"[0.7534204925509272, 0.44993819530284296, 0.35...",0.509453,"[0.7942307692307692, 0.44938271604938274, 0.28...",0.541759,"[0.7165991902834008, 0.4504950495049505, 0.458...",None,0.659762,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.631034,0.521311,"[0.7593939393939394, 0.43757934828607703, 0.36...",0.513032,"[0.7975811584977721, 0.44917463075586445, 0.29...",0.547997,"[0.7246963562753036, 0.4265676567656766, 0.492...",None,0.657514,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


best_seqsignet_network_grp_kfold_35["f1"].mean()

In [16]:
best_seqsignet_network_grp_kfold_35["precision"].mean()

0.5120608108461853

In [17]:
best_seqsignet_network_grp_kfold_35["recall"].mean()

0.5358542837125433

In [18]:
np.stack(best_seqsignet_network_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.75836367, 0.44601742, 0.35633603])

In [19]:
np.stack(best_seqsignet_network_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.78469397, 0.44456482, 0.30692364])

In [20]:
np.stack(best_seqsignet_network_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.73375747, 0.44774477, 0.42606061])

# history_length=80

In [21]:
shift = 3
window_size = 5
n = 26

In [22]:
(
    seqsignet_network_umap_kfold_80,
    best_seqsignet_network_umap_kfold_80,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_26_kfold_best_model.csv


In [23]:
seqsignet_network_umap_kfold_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.638123,0.511260,"[0.7656113854196922, 0.4130434782608695, 0.355...",0.509015,"[0.7689614935822637, 0.4578313253012048, 0.300...",0.524358,"[0.7622903412377097, 0.37623762376237624, 0.43...",None,0.678227,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.615517,0.500811,"[0.7467394601152563, 0.42379788101059496, 0.33...",0.492701,"[0.7850765306122449, 0.41867954911433175, 0.27...",0.520338,"[0.7119722382880278, 0.429042904290429, 0.42]",None,0.662171,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.621264,0.504750,"[0.7523980815347721, 0.4173535791757049, 0.344...",0.499015,"[0.7809583074051026, 0.4400731930466606, 0.276...",0.526967,"[0.7258530942741469, 0.39686468646864687, 0.45...",None,0.658157,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.627969,0.519001,"[0.7521626954014266, 0.44435643564356436, 0.36...",0.508899,"[0.7914404343660173, 0.42726580350342724, 0.30...",0.538005,"[0.7165991902834008, 0.4628712871287129, 0.434...",None,0.674053,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.623563,0.499286,"[0.7582205029013539, 0.40345110928512734, 0.33...",0.492185,"[0.7808764940239044, 0.40180032733224225, 0.29...",0.511562,"[0.7368421052631579, 0.4051155115511551, 0.392...",None,0.674534,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.643678,0.511655,"[0.772109340739658, 0.43267651888341546, 0.330...",0.508146,"[0.7806680461129175, 0.4305555555555556, 0.313...",0.515882,"[0.7637362637362637, 0.43481848184818483, 0.34...",None,0.667309,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.628352,0.514626,"[0.7533753375337534, 0.4345718901453958, 0.355...",0.505729,"[0.7827306733167082, 0.42563291139240506, 0.30...",0.530012,"[0.7261422787738577, 0.4438943894389439, 0.42]",None,0.660726,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.616858,0.512327,"[0.7410590943975441, 0.4392523364485982, 0.356...",0.501823,"[0.7896630683676807, 0.415929203539823, 0.2998...",0.534479,"[0.6980913823019086, 0.46534653465346537, 0.44]",None,0.671644,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.621456,0.512297,"[0.748324192565509, 0.43304620203602195, 0.355...",0.502401,"[0.7907276239536382, 0.4120715350223547, 0.304...",0.531260,"[0.7102371312897628, 0.45627062706270627, 0.42...",None,0.672286,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [24]:
best_seqsignet_network_umap_kfold_80["f1"].mean()

0.5145867126760523

In [25]:
best_seqsignet_network_umap_kfold_80["precision"].mean()

0.5061760879903806

In [26]:
best_seqsignet_network_umap_kfold_80["recall"].mean()

0.5356273643558688

In [27]:
np.stack(best_seqsignet_network_umap_kfold_80["f1_scores"]).mean(axis=0)

array([0.75421046, 0.43551048, 0.3540392 ])

In [28]:
np.stack(best_seqsignet_network_umap_kfold_80["precision_scores"]).mean(axis=0)

array([0.79040481, 0.43489877, 0.29322468])

In [29]:
np.stack(best_seqsignet_network_umap_kfold_80["recall_scores"]).mean(axis=0)

array([0.72132254, 0.43646865, 0.44909091])

## GRP

In [30]:
(
    seqsignet_network_grp_kfold_80,
    best_seqsignet_network_grp_kfold_80,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_26_kfold_best_model.csv


In [31]:
seqsignet_network_grp_kfold_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.609770,0.497510,"[0.7376334385806645, 0.4102127659574468, 0.344...",0.490253,"[0.7682430316316944, 0.4235500878734622, 0.278...",0.519323,"[0.7093695777906304, 0.3976897689768977, 0.450...",None,0.657514,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.631992,0.518703,"[0.7583497053045186, 0.4511894995898277, 0.346...",0.510092,"[0.7942386831275721, 0.44861337683523655, 0.28...",0.538574,"[0.7255639097744361, 0.4537953795379538, 0.436...",None,0.663616,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.632567,0.513589,"[0.7588452997779424, 0.42592592592592593, 0.35...",0.506355,"[0.7773733697300577, 0.43470790378006874, 0.30...",0.527436,"[0.7411798727588201, 0.4174917491749175, 0.423...",None,0.666506,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.641379,0.519888,"[0.7653482880755608, 0.4481012658227848, 0.346...",0.513706,"[0.7814948764315853, 0.4585492227979275, 0.301...",0.531749,"[0.7498554077501446, 0.4381188118811881, 0.407...",None,0.666185,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.636398,0.519464,"[0.7601904195180006, 0.443163097199341, 0.3550...",0.511559,"[0.7827818627450981, 0.44243421052631576, 0.30...",0.533041,"[0.7388663967611336, 0.4438943894389439, 0.416...",None,0.671162,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.630651,0.511715,"[0.7589697781747804, 0.4291155311045281, 0.347...",0.504662,"[0.7821417612764652, 0.44048653344917466, 0.29...",0.528180,"[0.7371312897628687, 0.4183168316831683, 0.429...",None,0.667470,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.624713,0.505451,"[0.7532506351815873, 0.4272242722427224, 0.335...",0.497849,"[0.7794618001855862, 0.42461287693561534, 0.28...",0.519538,"[0.728744939271255, 0.4298679867986799, 0.4]",None,0.664900,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.636782,0.519774,"[0.7612384638285203, 0.44584382871536526, 0.35...",0.512312,"[0.7843558282208589, 0.45384615384615384, 0.29...",0.535551,"[0.7394447657605553, 0.4381188118811881, 0.429...",None,0.662010,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.626437,0.518342,"[0.7540384029259373, 0.44370029300962743, 0.35...",0.510205,"[0.7970360824742269, 0.45029736618521665, 0.28...",0.545458,"[0.7154424522845575, 0.4372937293729373, 0.483...",None,0.657354,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [32]:
best_seqsignet_network_grp_kfold_80["f1"].mean()

0.5241674537377478

In [33]:
best_seqsignet_network_grp_kfold_80["precision"].mean()

0.5158555166569724

In [34]:
best_seqsignet_network_grp_kfold_80["recall"].mean()

0.5408855622404345

In [35]:
np.stack(best_seqsignet_network_grp_kfold_80["f1_scores"]).mean(axis=0)

array([0.76184031, 0.45014527, 0.36051678])

In [36]:
np.stack(best_seqsignet_network_grp_kfold_80["precision_scores"]).mean(axis=0)

array([0.78863811, 0.45154971, 0.30737873])

In [37]:
np.stack(best_seqsignet_network_grp_kfold_80["recall_scores"]).mean(axis=0)

array([0.73684211, 0.44884488, 0.4369697 ])

# history_length=110

In [11]:
shift = 3
window_size = 5
n = 36

In [12]:
(
    seqsignet_network_umap_kfold_110,
    best_seqsignet_network_umap_kfold_110,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_36_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_umap_focal_2_3_5_36_kfold_best_model.csv


In [13]:
seqsignet_network_umap_kfold_110

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.636973,0.515311,"[0.7650593580536421, 0.4157407407407407, 0.365...",0.514178,"[0.7756315007429421, 0.47362869198312235, 0.29...",0.536290,"[0.7547715442452284, 0.3704620462046205, 0.483...",None,0.669236,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.610345,0.507617,"[0.7401428127910585, 0.42733699921445406, 0.35...",0.497585,"[0.7989276139410187, 0.4077961019490255, 0.286...",0.535784,"[0.6894158473105841, 0.44884488448844884, 0.46...",None,0.659602,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.638506,0.512958,"[0.7647489386619821, 0.4292432663531424, 0.344...",0.507994,"[0.77438482063445, 0.4454303460514641, 0.30416...",0.522574,"[0.7553499132446501, 0.4141914191419142, 0.398...",None,0.673410,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.623563,0.513141,"[0.7479106518766145, 0.4461113304382156, 0.345...",0.503611,"[0.7880243355747678, 0.42770628311884934, 0.29...",0.531406,"[0.711683053788317, 0.4661716171617162, 0.4163...",None,0.669236,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.622989,0.502998,"[0.7546607009694257, 0.40545965475712564, 0.34...",0.495567,"[0.7791807822605482, 0.39483971853010164, 0.31...",0.514283,"[0.7316367842683632, 0.4166666666666667, 0.394...",None,0.678549,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.636015,0.510139,"[0.764280442804428, 0.4252280840936137, 0.3409...",0.506093,"[0.7805245703949352, 0.4094728800611154, 0.328...",0.515163,"[0.7486986697513013, 0.44224422442244227, 0.35...",None,0.671002,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.629502,0.517098,"[0.7561049140789871, 0.43509906995551967, 0.36...",0.507610,"[0.7896725440806045, 0.42664551942902457, 0.30...",0.535178,"[0.7252747252747253, 0.4438943894389439, 0.436...",None,0.662813,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.592912,0.496118,"[0.717040288446465, 0.4236783320923306, 0.3476...",0.486224,"[0.7829510441629579, 0.3860244233378562, 0.289...",0.521794,"[0.661364950838635, 0.46947194719471946, 0.434...",None,0.671644,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.612835,0.515650,"[0.7380915196001874, 0.4362670713201821, 0.372...",0.504289,"[0.802376910016978, 0.40379213483146065, 0.306...",0.544104,"[0.683342972816657, 0.4744224422442244, 0.4745...",None,0.666185,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [14]:
best_seqsignet_network_umap_kfold_110["f1"].mean()

0.5122063920877166

In [15]:
best_seqsignet_network_umap_kfold_110["precision"].mean()

0.504574592601258

In [16]:
best_seqsignet_network_umap_kfold_110["recall"].mean()

0.5315281815438148

In [17]:
np.stack(best_seqsignet_network_umap_kfold_110["f1_scores"]).mean(axis=0)

array([0.75049975, 0.4303473 , 0.35577213])

In [18]:
np.stack(best_seqsignet_network_umap_kfold_110["precision_scores"]).mean(axis=0)

array([0.78478681, 0.42784186, 0.30109511])

In [19]:
np.stack(best_seqsignet_network_umap_kfold_110["recall_scores"]).mean(axis=0)

array([0.71949104, 0.43509351, 0.44      ])

## GRP

In [20]:
(
    seqsignet_network_grp_kfold_110,
    best_seqsignet_network_grp_kfold_110,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_36_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_36_kfold_best_model.csv


In [21]:
seqsignet_network_grp_kfold_110

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.613985,0.501013,"[0.7408852213053264, 0.41495170096598066, 0.34...",0.493064,"[0.7698783910196445, 0.42258340461933275, 0.28...",0.520529,"[0.7139965297860035, 0.4075907590759076, 0.44]",None,0.658960,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.613602,0.504512,"[0.7419404125286478, 0.43223443223443225, 0.33...",0.495935,"[0.786524133462909, 0.42650602409638555, 0.274...",0.527965,"[0.70213996529786, 0.4381188118811881, 0.44363...",None,0.665703,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.635057,0.511647,"[0.7620164126611957, 0.41788478073946694, 0.35...",0.506052,"[0.7724301841948901, 0.4362657091561939, 0.309...",0.523078,"[0.7518796992481203, 0.400990099009901, 0.4163...",None,0.670681,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.636973,0.518748,"[0.760688836104513, 0.44733529164918173, 0.348...",0.511685,"[0.7815741305674192, 0.4551665243381725, 0.298...",0.532947,"[0.7408906882591093, 0.4397689768976898, 0.418...",None,0.663777,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.634866,0.516181,"[0.7607105538140022, 0.4444444444444444, 0.343...",0.508365,"[0.7861771058315334, 0.4401294498381877, 0.298...",0.529774,"[0.7368421052631579, 0.44884488448844884, 0.40...",None,0.672447,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.631418,0.512777,"[0.7600178757634439, 0.4285714285714286, 0.349...",0.505508,"[0.7837173579109062, 0.4393414211438475, 0.293...",0.529585,"[0.7377096587622903, 0.4183168316831683, 0.432...",None,0.666506,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.621073,0.504914,"[0.7481259370314842, 0.4269938650306749, 0.339...",0.496818,"[0.776774595267746, 0.4233576642335766, 0.2903...",0.520433,"[0.7215153267784846, 0.4306930693069307, 0.409...",None,0.661850,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.622222,0.507652,"[0.7483383685800604, 0.4405286343612335, 0.334...",0.499093,"[0.7833649588867805, 0.4280155642023346, 0.285...",0.523975,"[0.71631000578369, 0.4537953795379538, 0.40181...",None,0.660083,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.626628,0.516043,"[0.7521186440677966, 0.4407467532467533, 0.355...",0.506554,"[0.7888888888888889, 0.43370607028753994, 0.29...",0.536154,"[0.7186234817813765, 0.44801980198019803, 0.44...",None,0.660083,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [22]:
best_seqsignet_network_grp_kfold_110["f1"].mean()

0.5038014784222017

In [23]:
best_seqsignet_network_grp_kfold_110["precision"].mean()

0.49578871922868467

In [24]:
best_seqsignet_network_grp_kfold_110["recall"].mean()

0.5205197151679695

In [25]:
np.stack(best_seqsignet_network_grp_kfold_110["f1_scores"]).mean(axis=0)

array([0.74488423, 0.42798057, 0.33853964])

In [26]:
np.stack(best_seqsignet_network_grp_kfold_110["precision_scores"]).mean(axis=0)

array([0.7761339 , 0.42346957, 0.28776268])

In [27]:
np.stack(best_seqsignet_network_grp_kfold_110["recall_scores"]).mean(axis=0)

array([0.71621361, 0.43261826, 0.41272727])